In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from padertorch.contrib.examples.scvae.model import SCVAE
from padertorch.contrib.examples.scvae.data import Transform
from paderbox.database.timit import Timit
from paderbox.database.librispeech import LibriSpeech
import numpy as np
import torch
from paderbox.visualization import plot
import IPython.display as ipd

In [ ]:
storage_dir = '/net/vol/ebbers/exp/scvae-training/2019-06-16-21-00-22'

In [ ]:
scvae = SCVAE.from_storage_dir(storage_dir=storage_dir, config_name='1/config.json', checkpoint_name='ckpt_latest.pth')

In [ ]:
db = Timit()
transform = Transform(storage_dir=storage_dir)
transform.initialize_norm()
transform.initialize_labels()
ds = db.get_dataset('train')

#def fix_speaker_id(example):
#    example['speaker_id'] = example['speaker_id'].split('-')[0]
#    return example
#ds = ds.map(fix_speaker_id)

ds = ds.map(
    transform.read_audio
).map(
    transform.extract_features
).map(
    transform.normalize
).map(
    transform.encode_labels
)
mean, std = transform.moments


In [ ]:
example1 = ds[100]
example2 = ds[3603]
print(example1['log_mel'].shape)
print(example2['log_mel'].shape)
print(example1['speaker_id'], example2['speaker_id'])

In [ ]:
spec1 = transform.mel_transform.inverse(std*example1['log_mel'] + mean)
print(spec1.shape)
spec2 = transform.mel_transform.inverse(std*example2['log_mel'] + mean)
print(spec2.shape)
plot.spectrogram(spec1[0])
plot.spectrogram(spec2[0])

In [ ]:
audio1 = transform.stft.griffin_lim(np.sqrt(spec1))
audio2 = transform.stft.griffin_lim(np.sqrt(spec2))
print(audio1.shape)
print(audio2.shape)

In [ ]:
ipd.Audio(audio1, rate=16000)

In [ ]:
ipd.Audio(audio2, rate=16000)

In [ ]:
vc12 = scvae.conversion(
    torch.from_numpy(np.moveaxis(example1['log_mel'], 1, 2).astype(np.float32)),
    torch.Tensor(example2['speaker_id'][None]).long()
).transpose(1, 2).cpu().data.numpy()
vc21 = scvae.conversion(
    torch.from_numpy(np.moveaxis(example2['log_mel'], 1, 2).astype(np.float32)),
    torch.Tensor(example1['speaker_id'][None]).long()
).transpose(1, 2).cpu().data.numpy()

In [ ]:
spec12 = transform.mel_transform.inverse(std*vc12 + mean)
spec21 = transform.mel_transform.inverse(std*vc21 + mean)
print(spec12.shape)
print(spec21.shape)
plot.spectrogram(spec12[0])
plot.spectrogram(spec21[0])

In [ ]:
audio12 = transform.stft.griffin_lim(np.sqrt(spec12))
audio21 = transform.stft.griffin_lim(np.sqrt(spec21))
print(audio12.shape)
print(audio21.shape)

In [ ]:
ipd.Audio(audio12, rate=16000)

In [ ]:
ipd.Audio(audio21, rate=16000)